In [54]:
from tinybig.config import rpn_config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'composite_configs'
config_obj = rpn_config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'cpu', 'random_seed': 123, 'data_configs': {'data_class': 'tinybig.data.composite_function', 'data_parameters': {'name': 'elementary_function', 'train_batch_size': 100, 'test_batch_size': 100, 'equation_index': 3}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 2000, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.003, 'weight_decay': 0.0001}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.999}}, 'loss_configs': {'loss_class': 'torch.nn.MSELoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'name': 'reconciled_polynomial_network', 'depth': 3, 'depth_alloc': [2, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_parame

In [55]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  cpu ; random_seed:  123


In [56]:
print('instantiating objects from config...')
object_dict = config_obj.instantiate_object_from_config(config['configurations'])
data_obj, model_obj, learner_obj, metric_obj, result_obj = [object_dict[name] for name in ['data', 'model', 'learner', 'metric', 'result']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  2441


In [57]:
print('loading dataset...')
data_loader = data_obj.load()
print(data_loader['str_equation'])

loading dataset...
C.3,3,f,exp(x+y)+ln(x+y),2,x,0,1,y,0,1


In [58]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device,
                                     metric=metric_obj, disable_tqdm=True, display_step=100)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...
Epoch: 0, Test Loss: 11.524025630950927, Test Score: 11.524025590011567, Time Cost: 0.013231277465820312
Epoch: 100, Test Loss: 0.00019555933140509297, Test Score: 0.00019555932755814802, Time Cost: 0.013195037841796875
Epoch: 200, Test Loss: 7.714675048191566e-05, Test Score: 7.714674850153402e-05, Time Cost: 0.012295722961425781
Epoch: 300, Test Loss: 0.00013800916640320792, Test Score: 0.00013800916351929066, Time Cost: 0.012665748596191406
Epoch: 400, Test Loss: 2.8712480525427963e-05, Test Score: 2.8712480293826946e-05, Time Cost: 0.01266789436340332
Epoch: 500, Test Loss: 3.685379961098079e-05, Test Score: 3.685379997127891e-05, Time Cost: 0.013297080993652344
Epoch: 600, Test Loss: 3.0447333665506447e-05, Test Score: 3.044733344584222e-05, Time Cost: 0.013369083404541016
Epoch: 700, Test Loss: 1.1872851132466167e-05, Test Score: 1.1872851446747989e-05, Time Cost: 0.012234210968017578
Epoch: 800, Test Loss: 1.2467668420867995e-05, Test Score: 1.2467668700378854e

In [59]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [60]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
mse 2.310927682198649e-06


In [61]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
